In [19]:
#기본 라이브러리 불러오기
import pandas as pd
import scipy
import numpy as np
import statsmodels.api as sm
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from yellowbrick.cluster import intercluster_distance
from sklearn.decomposition import PCA
from scipy.stats import gaussian_kde, skew
from sklearn.preprocessing import OneHotEncoder, StandardScaler

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)

#데이터 불러오기 (챌그마 매치데이터_클러스터링용)
df = pd.read_csv('C:/Users/alaru/OneDrive/바탕 화면/부트캠프/data/final_project/LOL_match data_C+GM+M.csv')

#정글러만 모아놓은 새로운 df 생성
dfj = df[df['teamPosition']=='JUNGLE']

#경기시간이 600초를 넘는 경우/경험치 0 이상 + 14시즌 필터링 → df2로 정의
mask = (dfj['timePlayed'] > 600)
mask2 = (dfj['champExperience'] > 0)
dfj_over600 = dfj[mask]
dfj_over600_2 = dfj_over600[mask2]
dfj_over600_2['season'] = dfj_over600_2['gameType'].str[:2]
season_mask = (dfj_over600_2['season'] == '14')
df2 = dfj_over600_2[season_mask]

#파생변수 생성 (kda, 시간당 골드, 시간당 경험치 등등)
df2['kda'] = (df2['kills'] + df2['assists'])/(df2['deaths']+1)
df2['goldpertime'] = (df2['goldEarned'])/(df2['timePlayed'])
df2['exppertime'] = (df2['champExperience'])/(df2['timePlayed'])
df2['junglepertime'] = (df2['neutralMinionsKilled'])/(df2['timePlayed'])
df2['deathduration'] = (df2['totalTimeSpentDead'])/(df2['deaths']+1)
df2['exppergold'] = (df2['champExperience'])/(df2['goldEarned'])
df2['goldperexp'] = (df2['goldEarned'])/(df2['champExperience'])
df2['dealpergold'] = (df2['totalDamageDealtToChampions'])/(df2['goldEarned'])
df2['dealperlevel'] = (df2['totalDamageDealtToChampions'])/(df2['champLevel'])
df2['tankpergold'] = (df2['totalDamageTaken'])/(df2['goldEarned'])
df2['vspertime'] = (df2['visionScore'])/(df2['timePlayed'])
df2['kapertime'] = (df2['kills'] + df2['assists'])/(df2['timePlayed'])
df2['kaperlevel'] = (df2['kills'] + df2['assists'])/(df2['champLevel'])
df2['dealtankratio'] = (df2['totalDamageDealtToChampions'])/(df2['totalDamageTaken'])
df2['counterratio'] = (df2['totalEnemyJungleMinionsKilled'])/(df2['totalAllyJungleMinionsKilled'])
df2['allyjungleratio'] = (df2['totalAllyJungleMinionsKilled'])/(df2['neutralMinionsKilled'])
df2['enemyjungleratio'] = (df2['totalEnemyJungleMinionsKilled'])/(df2['neutralMinionsKilled'])
df2['madamratio'] = (df2['magicDamageDealtToChampions'])/(df2['totalDamageDealtToChampions'])
df2['phydamratio'] = (df2['physicalDamageDealtToChampions'])/(df2['totalDamageDealtToChampions'])
df2['dagulperexp'] = (df2['pickKillWithAlly'])/(df2['champExperience'])
df2['killdamratio'] = (df2['kills'])/(df2['totalDamageDealtToChampions'])
df2['damperdeath'] = (df2['totalDamageTaken'])/(df2['deaths']+1)

C:\Users\alaru\AppData\Local\Temp\ipykernel_27396\2651696516.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfj_over600_2 = dfj_over600[mask2]
C:\Users\alaru\AppData\Local\Temp\ipykernel_27396\2651696516.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfj_over600_2['season'] = dfj_over600_2['gameType'].str[:2]
C:\Users\alaru\AppData\Local\Temp\ipykernel_27396\2651696516.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['kda'] = (df2['kills'] + df2['assists']

In [18]:
#이상치(무한대) 확인
infinite_values_count = df2.apply(lambda x: (x == float('inf')).sum())
infinite_columns = infinite_values_count[infinite_values_count > 0]

#무한대를 없애는 함수
def remove_infinite_rows(df):
    return df[~df.isin([np.inf, -np.inf]).any(axis=1)]

df3 = remove_infinite_rows(df2)

In [5]:
#matchID당 정글러 숫자가 2가 아닌 경우는 제외(filtered_dfj)
dfj_count = df3.groupby('matchId')['gameType'].count()
invalid_gameType_matchIds = dfj_count[dfj_count != 2].index.tolist()
filtered_dfj = df3[~df3['matchId'].isin(invalid_gameType_matchIds)]

In [20]:
filtered_dfj.isnull().sum()

matchId                                          0
gameType                                         0
assists                                          0
baronKills                                       0
champExperience                                  0
champLevel                                       0
championName                                     0
damageDealtToBuildings                           0
damageDealtToObjectives                          0
damageDealtToTurrets                             0
damageSelfMitigated                              0
deaths                                           0
detectorWardsPlaced                              0
dragonKills                                      0
goldEarned                                       0
inhibitorTakedowns                               0
item0                                            0
item1                                            0
item2                                            0
item3                          

In [25]:
#Null값을 0으로 채우기
filtered_dfj['controlWardTimeCoverageInRiverOrEnemyHalf'] = filtered_dfj['controlWardTimeCoverageInRiverOrEnemyHalf'].fillna(0)
filtered_dfj['laningPhaseGoldExpAdvantage'] = filtered_dfj['laningPhaseGoldExpAdvantage'].fillna(0)
filtered_dfj['counterratio'] = filtered_dfj['counterratio'].fillna(0)
filtered_dfj['allyjungleratio'] = filtered_dfj['allyjungleratio'].fillna(0)
filtered_dfj['enemyjungleratio'] = filtered_dfj['enemyjungleratio'].fillna(0)
filtered_dfj['madamratio'] = filtered_dfj['madamratio'].fillna(0)
filtered_dfj['phydamratio'] = filtered_dfj['phydamratio'].fillna(0)
filtered_dfj['killdamratio'] = filtered_dfj['killdamratio'].fillna(0)

In [27]:
#CSV로 추출
filtered_dfj.to_csv('cleaned_data.csv', index=False)